In [1]:
# common imports

import sys
sys.path.append("../datasets/ARID_supporting_scripts")

import os
import random
import mapper
import datasets
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from tensorflow import keras

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

2025-03-21 20:17:39.323177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-21 20:17:39.323215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-21 20:17:39.324512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 20:17:39.331360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 20:17:39.917846: W tensorflow/compiler/tf2

# Model Setup

In [2]:
# Loading the dataset  

evaluation_set = datasets.load_from_disk('../datasets/ARID_supporting_scripts/5_1_training_set')['test']

In [3]:
lbl_ = evaluation_set.features['label'].names
label2id = {lbl: idx for idx, lbl in enumerate(lbl_)}
id2label = {val: key for key, val in label2id.items()}

In [4]:
def preprocess_function(dataset):
    return tokenizer(dataset['Requirement Sentences'], padding = 'max_length', max_length = 256, truncation = True, return_tensors = 'tf')

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

def forward_pass_with_label(batch):
    inputs = {'input_ids': tf.convert_to_tensor(batch['input_ids']),
             'attention_mask': tf.convert_to_tensor(batch['attention_mask'])}
    true_labels = tf.convert_to_tensor(batch['label'])
    
    with tf.GradientTape() as tape:
        output = model(**inputs)
        predicted_labels = tf.argmax(output.logits, axis = -1).numpy()
        probas = tf.nn.softmax(output.logits, axis = -1).numpy()
        loss = tf.keras.losses.sparse_categorical_crossentropy(true_labels, output.logits)

    loss = loss.numpy()

    return {"loss": loss, 
            "y_preds": [id2label[lbl] for lbl in predicted_labels],
            "y_probas": [probas[i][predicted_labels[i]] for i in range(len(predicted_labels))]}

# Benchmark: All Fine-tuned Models 

In [6]:
tuned_benchmark_models_path = Path('./models/benchmark/tuned/')

tuned_benchmark_models = [folder for folder in tuned_benchmark_models_path.iterdir() if folder.is_dir() and not folder.name.startswith('.')]
tuned_benchmark_models

[PosixPath('models/benchmark/tuned/reqseek_gpt2_finetuned'),
 PosixPath('models/benchmark/tuned/reqseek_roberta-base_finetuned'),
 PosixPath('models/benchmark/tuned/reqseek_bert-base-cased_finetuned'),
 PosixPath('models/benchmark/tuned/reqseek_all-MiniLM-L6-v2_finetuned')]

In [7]:
all_tuned_models = {}

for mdl in tuned_benchmark_models:
    split_mdl_id = mdl.name.split('_')
    model_name = split_mdl_id[1]
    all_tuned_models[model_name] = mdl

all_tuned_models

{'gpt2': PosixPath('models/benchmark/tuned/reqseek_gpt2_finetuned'),
 'roberta-base': PosixPath('models/benchmark/tuned/reqseek_roberta-base_finetuned'),
 'bert-base-cased': PosixPath('models/benchmark/tuned/reqseek_bert-base-cased_finetuned'),
 'all-MiniLM-L6-v2': PosixPath('models/benchmark/tuned/reqseek_all-MiniLM-L6-v2_finetuned')}

In [8]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

all_tuned_models_prediction = {}

for mdl_id in list(all_tuned_models.keys()):
    model_path = all_tuned_models[mdl_id]
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
    if mdl_id == 'gpt2':
        if model.config.pad_token_id is None:
            print("Pad token ID is not set. Setting now...")
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
            model.config.pad_token_id = tokenizer.pad_token_id
            tokenizer.padding_side = "left"
            print("New Pad Token ID set:", model.config.pad_token_id)
    evaluation_set_encoded = evaluation_set.map(preprocess_function, batched = True)
    evaluation_set_predicted = evaluation_set_encoded.map(forward_pass_with_label, batched = True, batch_size = 8)        
    all_tuned_models_prediction[mdl_id] = evaluation_set_predicted

2025-03-21 20:17:42.083026: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-21 20:17:42.084915: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-21 20:17:42.088327: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

2025-03-21 20:17:45.698530: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at models/benchmark/tuned/reqseek_roberta-base_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Some layers from the model checkpoint at models/benchmark/tuned/reqseek_bert-base-cased_finetuned were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at models/benchmark/tuned/reqseek_bert-base-cased_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further trainin

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Some layers from the model checkpoint at models/benchmark/tuned/reqseek_all-MiniLM-L6-v2_finetuned were not used when initializing TFBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at models/benchmark/tuned/reqseek_all-MiniLM-L6-v2_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further train

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(y_true, y_pred, average = 'macro'):
    print(f"Precision: {precision_score(y_true, y_pred, average = average)}")
    print(f"Recall: {recall_score(y_true, y_pred, average = average)}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average = average)}")

In [10]:
print('Fine-tuned: Models Performance Benchmark\n')
for key, value in  all_tuned_models_prediction.items():
    y_true = mapper.map(value['signal_keyword'])
    y_pred = mapper.map(value['y_preds'])
    print(key)
    evaluate(y_true, y_pred)
    print('\n')

Fine-tuned: Models Performance Benchmark

gpt2
Precision: 0.6285157843485584
Recall: 0.5295238095238095
F1-Score: 0.4512597059471009


roberta-base
Precision: 0.9541119708891043
Recall: 0.9517460317460317
F1-Score: 0.9528933151427276


bert-base-cased
Precision: 0.9379420599040573
Recall: 0.9319047619047618
F1-Score: 0.9347329572480018


all-MiniLM-L6-v2
Precision: 0.9430809116003013
Recall: 0.9322222222222222
F1-Score: 0.9369875105206669




# Benchmark: All Untuned Baseline Models 

In [11]:
untuned_benchmark_models_path = Path('./training_scripts_and_models/models/untuned/')

untuned_benchmark_models = [folder for folder in untuned_benchmark_models_path.iterdir() if folder.is_dir() and not folder.name.startswith('.')]
untuned_benchmark_models

[PosixPath('models/benchmark/untuned/untuned_gpt2'),
 PosixPath('models/benchmark/untuned/untuned_roberta-base'),
 PosixPath('models/benchmark/untuned/untuned_all-MiniLM-L6-v2'),
 PosixPath('models/benchmark/untuned/untuned_bert-base-cased')]

In [12]:
all_untuned_models = {}

for mdl in untuned_benchmark_models:
    split_mdl_id = mdl.name.split('_')
    model_name = split_mdl_id[1]
    all_untuned_models[model_name] = mdl

all_untuned_models

{'gpt2': PosixPath('models/benchmark/untuned/untuned_gpt2'),
 'roberta-base': PosixPath('models/benchmark/untuned/untuned_roberta-base'),
 'all-MiniLM-L6-v2': PosixPath('models/benchmark/untuned/untuned_all-MiniLM-L6-v2'),
 'bert-base-cased': PosixPath('models/benchmark/untuned/untuned_bert-base-cased')}

In [13]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

all_untuned_models_prediction = {}

for mdl_id in list(all_untuned_models.keys()):
    model_path = all_untuned_models[mdl_id]
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
    if mdl_id == 'gpt2':
        if model.config.pad_token_id is None:
            print("Pad token ID is not set. Setting now...")
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
            model.config.pad_token_id = tokenizer.pad_token_id
            tokenizer.padding_side = "left"
            print("New Pad Token ID set:", model.config.pad_token_id)
    evaluation_set_encoded = evaluation_set.map(preprocess_function, batched = True)
    evaluation_set_predicted = evaluation_set_encoded.map(forward_pass_with_label, batched = True, batch_size = 8)        
    all_untuned_models_prediction[mdl_id] = evaluation_set_predicted

All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFGPT2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pad token ID is not set. Setting now...
New Pad Token ID set: 50256


/home/kasra/metal-engine/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [14]:
print('Untune: Models Performance Benchmark\n')
for key, value in  all_untuned_models_prediction.items():
    y_true = mapper.map(value['signal_keyword'])
    y_pred = mapper.map(value['y_preds'])
    print(key)
    evaluate(y_true, y_pred)
    print('\n')

Untune: Models Performance Benchmark

gpt2
Precision: 0.22680812184075774
Recall: 0.3337301587301587
F1-Score: 0.26973085615332687


roberta-base
Precision: 0.14583333333333334
Recall: 0.3333333333333333
F1-Score: 0.2028985507246377


all-MiniLM-L6-v2
Precision: 0.22465580704858357
Recall: 0.32261904761904764
F1-Score: 0.16988707593550076


bert-base-cased
Precision: 0.14583333333333334
Recall: 0.3333333333333333
F1-Score: 0.2028985507246377




/home/kasra/metal-engine/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kasra/metal-engine/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kasra/metal-engine/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/k